In [12]:
import pandas as pd
import os
from pathlib import Path
from pathlib import Path
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
%matplotlib inline

In [2]:
file = Path('../../../data/cleandata/CMC_df.csv')
tickers_df = pd.read_csv(file)
tickers_df.head()

,rank_cnc,Name,ticker,market_cap,price,volume24,pct1h,pct24h,pct7d
0,1,Bitcoin,BTC,"156,629,905,950","8,678.58","19,216,389,896",-0.28,-1.35,-6.14
1,2,Ethereum,ETH,"20,155,411,771",185.67,"7,906,136,399",-0.10,-1.20,-0.96
2,3,XRP,XRP,"11,612,597,542",0.268199,"1,265,885,288",-0.57,-1.75,-7.36
3,4,Bitcoin Cash,BCH,"5,016,918,192",276.97,"1,842,963,894",-0.75,-3.01,-5.39
4,5,Tether,USDT,"4,135,369,515",1.01,"22,054,927,684",-0.08,0.36,0.34


In [3]:
tickers_dict = tickers_df.set_index('ticker').to_dict()['Name']

In [4]:
file2 = Path('./tickers.csv')
tickers_df = pd.read_csv(file2)
tickers_df.dropna(inplace=True)
tickers_df.sort_values(by=['End'])
tickers_df = tickers_df.iloc[0:720]
tickers_dict2 = tickers_df.set_index('Ticker').to_dict()['Name']

In [5]:
len(tickers_dict)

2270

In [6]:
len(tickers_dict2)

703

In [7]:
new_dict = {}
for key, value in tickers_dict.items():
    if key not in tickers_dict2:
        new_dict[key] = value

In [8]:
len(new_dict)

1954

In [15]:
dict_group1 = {k: new_dict[k] for k in sorted(new_dict.keys())[:1000]}

In [16]:
len(dict_group1)

1000

In [13]:
import webhoseio
api_key = os.getenv("WEBHOSE_API_2")
webhoseio.config(token=api_key)

In [23]:
articles_dict = {}
for key, value in dict_group1.items():
    articles_dict[key] = webhoseio.query("filterWebContent", {"q": f"crypto AND ({key} OR {value})", "language":"english"})

Exception: Failed to execute query: internal error.
Warning: unbalanced parentheses.


In [24]:
len(articles_dict)

246

In [27]:
sentiment_df = pd.DataFrame()
analyzer = SentimentIntensityAnalyzer()

In [28]:
for key in articles_dict:
    sentiments = []
    for article in articles_dict[key]["posts"]:
        try:
            sentiment = analyzer.polarity_scores(article["text"])
      
            sentiments.append({
                "ticker": key,
                "compound": sentiment["compound"],
                "positive": sentiment["pos"],
                "negative": sentiment["neg"],
                "neutral": sentiment["neu"]
            
            })
        
        except AttributeError:
            pass
      
    sentiment_df = sentiment_df.append(pd.DataFrame(sentiments))

In [29]:
sentiment_df.head()

,ticker,compound,positive,negative,neutral
0,0xBTC,0.2263,0.006,0.000,0.994
0,1337,0.9993,0.102,0.073,0.825
1,1337,0.9824,0.049,0.024,0.927
2,1337,0.9569,0.179,0.056,0.765
3,1337,0.9824,0.049,0.024,0.927


In [30]:
sentiment_df = sentiment_df.set_index("ticker")

In [31]:
sentiment_avg = sentiment_df.groupby(['ticker']).mean()
sentiment_avg.head()

,compound,positive,negative,neutral
ticker,,,,
0xBTC,0.226300,0.006000,0.000000,0.994000
1337,0.433034,0.086242,0.050839,0.862742
1SG,-0.067967,0.015333,0.018667,0.966000
1X2,0.541983,0.097120,0.041350,0.861500
3DC,-0.571900,0.005000,0.019000,0.976000


In [32]:
sentiment_count = sentiment_df.groupby(['ticker']).count()
sentiment_count.head()

,compound,positive,negative,neutral
ticker,,,,
0xBTC,1,1,1,1
1337,62,62,62,62
1SG,3,3,3,3
1X2,100,100,100,100
3DC,1,1,1,1


In [33]:
sentiment_count = sentiment_count.drop(columns=['negative','neutral','positive'])
sentiment_count = sentiment_count.rename(columns={'compound':'no_of_posts'})
sentiment_count.head()

,no_of_posts
ticker,
0xBTC,1
1337,62
1SG,3
1X2,100
3DC,1


In [34]:
ticker_sentiments = pd.concat([sentiment_avg, sentiment_count], axis=1)
ticker_sentiments.head()

,compound,positive,negative,neutral,no_of_posts
ticker,,,,,
0xBTC,0.226300,0.006000,0.000000,0.994000,1
1337,0.433034,0.086242,0.050839,0.862742,62
1SG,-0.067967,0.015333,0.018667,0.966000,3
1X2,0.541983,0.097120,0.041350,0.861500,100
3DC,-0.571900,0.005000,0.019000,0.976000,1


In [35]:
len(ticker_sentiments)

196

In [36]:
file = Path(f"./ticker_sentiments.csv")
ticker_sentiments.to_csv(file, mode='a', header=False)